<a href="https://colab.research.google.com/github/Letrus/Seattle-Urban-Heat-Island-Project/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install earthaccess geopandas pandas numpy requests rasterio rioxarray shapely fiona pyogrio tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.0
    Uninstalling wrapt-2.0.0:
      Successfully uninstalled wrapt-2.0.0
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.5.0
    Uninstalling tenacity-8.5.0:
      Successfully uninstal

In [3]:
#import earthaccess
#earthaccess.login()


Enter your Earthdata Login username: letrus
Enter your Earthdata password: ··········


In [5]:
import os, sys, zipfile, tarfile, io, shutil, json, warnings
from pathlib import Path
from datetime import datetime
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
from shapely.ops import unary_union

import requests
from tqdm import tqdm

import rasterio
from rasterio.mask import mask as rio_mask
import rioxarray as rxr

import earthaccess  # NASA Earthdata

# ---- where to store project data (Colab local by default) ----
USE_DRIVE = False  # set True to save in your Google Drive

if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_DIR = Path("/content/drive/MyDrive/UHI_Project")
else:
    BASE_DIR = Path("/content/UHI_Project")

DATA_DIR = BASE_DIR / "data"
RAW_DIR  = DATA_DIR / "raw"
OUT_DIR  = DATA_DIR / "processed"
TMP_DIR  = DATA_DIR / "tmp"
for d in [DATA_DIR, RAW_DIR, OUT_DIR, TMP_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("BASE_DIR:", BASE_DIR)


BASE_DIR: /content/UHI_Project


In [6]:
# Option A: Use a Seattle bbox (quick start)
SEATTLE_BBOX = (-123.0, 47.1, -121.5, 48.3)  # (minx, miny, maxx, maxy) in WGS84

# Option B: Upload a boundary file (GeoPackage/GeoJSON) and set CUSTOM_BOUNDARY_FILE to its path
CUSTOM_BOUNDARY_FILE = None  # e.g., "/content/seattle_boundary.gpkg"

TARGET_EPSG = 3857  # for quick mapping. Use 26910 (UTM10N) if you prefer analysis in meters.

def get_study_geom():
    if CUSTOM_BOUNDARY_FILE and Path(CUSTOM_BOUNDARY_FILE).exists():
        g = gpd.read_file(CUSTOM_BOUNDARY_FILE)
        if g.empty:
            raise ValueError("Boundary file has no geometry.")
        g = g.to_crs(TARGET_EPSG)
        geom = unary_union(g.geometry.values)
        return gpd.GeoDataFrame(geometry=[geom], crs=g.crs)
    # fallback to bbox
    minx, miny, maxx, maxy = SEATTLE_BBOX
    g = gpd.GeoDataFrame(geometry=[box(minx, miny, maxx, maxy)], crs=4326).to_crs(TARGET_EPSG)
    return g

study = get_study_geom()
(OUT_DIR / "boundaries").mkdir(parents=True, exist_ok=True)
study.to_file(OUT_DIR / "boundaries" / "seattle_study_area.gpkg", driver="GPKG")
study


,geometry
0,"POLYGON ((-13525318.131 5958411.92, -13525318...."


In [9]:
from google.colab import files
uploaded = files.upload()  # pick a GeoPackage (.gpkg) or GeoJSON
uploaded


{}

In [10]:
def download_file(url: str, dest: Path, headers=None):
    dest.parent.mkdir(parents=True, exist_ok=True)
    with requests.get(url, stream=True, headers=headers) as r:
        r.raise_for_status()
        total = int(r.headers.get("content-length", 0))
        with open(dest, "wb") as f, tqdm(total=total, unit="B", unit_scale=True, desc=dest.name) as pbar:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))
    return dest

def unzip_to(src_zip: Path, dest_dir: Path):
    with zipfile.ZipFile(src_zip, "r") as z:
        z.extractall(dest_dir)
    return dest_dir

def untar_to(src_tar: Path, dest_dir: Path):
    with tarfile.open(src_tar, "r:*") as t:
        t.extractall(dest_dir)
    return dest_dir

def save_clip_raster(src_path: Path, geom_gdf: gpd.GeoDataFrame, out_path: Path):
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with rasterio.open(src_path) as src:
        geom = geom_gdf.to_crs(src.crs).geometry
        out_image, out_transform = rio_mask(src, geom, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        with rasterio.open(out_path, "w", **out_meta) as dst:
            dst.write(out_image)
    return out_path


In [12]:
# Choose which MODIS to fetch:
USE_MOD11_MYD11 = True   # Terra/Aqua classic 8-day LST
USE_MOD21       = False  # Emissivity-corrected alternative

START_DATE = "2022-06-01"
END_DATE   = "2022-09-30"

def earthdata_login_interactive():
    print("Sign into NASA Earthdata in the popup…")
    earthaccess.login()  # Colab-friendly interactive auth

def modis_search_and_download(product_short: str, start: str, end: str, bbox_wsen, outdir: Path):
    print(f"\nSearching {product_short} {start}→{end} over {bbox_wsen}…")
    results = earthaccess.search_data(
        short_name=product_short,
        temporal=(start, end),
        bounding_box=bbox_wsen
    )
    if not results:
        print(f"No granules found for {product_short}.")
        return []
    print(f"Found {len(results)} granules. Downloading…")
    outdir.mkdir(parents=True, exist_ok=True)


    files = earthaccess.download(results, str(outdir))

    return [Path(f) for f in files]

def fetch_modis(study_geom):
    bbox = study_geom.to_crs(4326).total_bounds
    bbox_wsen = (bbox[0], bbox[1], bbox[2], bbox[3])
    earthdata_login_interactive()

    lst_out = RAW_DIR / "MODIS"
    downloaded = []
    if USE_MOD11_MYD11:
        downloaded += modis_search_and_download("MOD11A2", START_DATE, END_DATE, bbox_wsen, lst_out / "MOD11A2")
        downloaded += modis_search_and_download("MYD11A2", START_DATE, END_DATE, bbox_wsen, lst_out / "MYD11A2")
    if USE_MOD21:
        downloaded += modis_search_and_download("MOD21A2", START_DATE, END_DATE, bbox_wsen, lst_out / "MOD21A2")

    if not downloaded:
        print("WARNING: No MODIS files downloaded. Check dates/product/bbox.")
        return []
    print(f"Downloaded {len(downloaded)} MODIS granules.")
    return downloaded

modis_files = fetch_modis(study)


Sign into NASA Earthdata in the popup…

Searching MOD11A2 2022-06-01→2022-09-30 over (np.float64(-123.00000000000001), np.float64(47.099999999999994), np.float64(-121.5), np.float64(48.3))…
Found 17 granules. Downloading…


QUEUEING TASKS | :   0%|          | 0/26 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/26 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/26 [00:00<?, ?it/s]


Searching MYD11A2 2022-06-01→2022-09-30 over (np.float64(-123.00000000000001), np.float64(47.099999999999994), np.float64(-121.5), np.float64(48.3))…
Found 17 granules. Downloading…


QUEUEING TASKS | :   0%|          | 0/26 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/26 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/26 [00:00<?, ?it/s]

Downloaded 52 MODIS granules.


In [14]:
def fetch_egrid(study_geom):
    # EPA often updates these URLs each release; these are current for eGRID 2022.
    egrid_xlsx_url = "https://www.epa.gov/system/files/documents/2025-06/egrid2023_data_rev2.xlsx"
    egrid_geo_zip  = "https://www.epa.gov/system/files/other-files/2025-01/egrid2023_subregions.zip"

    dest_xlsx = RAW_DIR / "egrid" / "egrid2022_data.xlsx"
    dest_geo  = RAW_DIR / "egrid" / "egrid2022_geospatial_data.zip"
    dest_geo_dir = RAW_DIR / "egrid" / "geospatial"

    print("\nDownloading eGRID 2022 (tabular)…")
    download_file(egrid_xlsx_url, dest_xlsx)

    print("Downloading eGRID 2022 (geospatial)…")
    download_file(egrid_geo_zip, dest_geo)
    unzip_to(dest_geo, dest_geo_dir)

    shp_candidates = list(dest_geo_dir.rglob("*plant*.shp"))
    if not shp_candidates:
        raise FileNotFoundError("Plant shapefile not found inside eGRID geospatial zip.")
    plants = gpd.read_file(shp_candidates[0])
    if plants.crs is None:
        plants.set_crs(epsg=4326, inplace=True)
    plants = plants.to_crs(study_geom.crs)

    # keep plants within ~50 km of study area
    sel = plants[plants.geometry.notnull()].copy()
    sel = gpd.overlay(sel, study_geom.buffer(50_000), how="intersection")
    out = OUT_DIR / "egrid_plants_seattle.gpkg"
    sel.to_file(out, driver="GPKG")
    print("Saved plants near Seattle →", out)
    return out

egrid_out = fetch_egrid(study)


egrid2022_data.xlsx: 100%|██████████| 21.2M/21.2M [01:03<00:00, 336kB/s]


egrid2022_geospatial_data.zip: 100%|██████████| 57.3M/57.3M [02:35<00:00, 367kB/s]


FileNotFoundError: Plant shapefile not found inside eGRID geospatial zip.

In [17]:
import re
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path

def _normalize(s: str) -> str:
    return re.sub(r'[^a-z0-9]', '', s.lower())

def _guess_lat_lon_columns(columns):
    """Return (lat_col, lon_col) or (None, None). Tries many variants."""
    # Build normalized map
    norm_map = {_normalize(c): c for c in columns}
    keys = list(norm_map.keys())

    # Candidates for latitude
    lat_candidates = [
        r'(^|.*)(plant)?lat(itude)?(.*)$',        # lat, latitude, plant latitude
        r'(^|.*)ycoord(.*)$',                     # ycoord
        r'(^|.*)y$'                               # y
    ]
    # Candidates for longitude
    lon_candidates = [
        r'(^|.*)(plant)?(lon(g|gitude)?|lng)(.*)$',  # lon, long, longitude, lng, plant longitude
        r'(^|.*)xcoord(.*)$',                         # xcoord
        r'(^|.*)x$'                                   # x
    ]

    lat_match, lon_match = None, None
    # Prefer names that also include "plant"
    def pick(cands, prefer_plant=True):
        # try with 'plant' first
        for patt in cands:
            for k in keys:
                if re.match(patt, k):
                    if prefer_plant and 'plant' in k:
                        return norm_map[k]
        # then any match
        for patt in cands:
            for k in keys:
                if re.match(patt, k):
                    return norm_map[k]
        return None

    lat_match = pick(lat_candidates, prefer_plant=True)
    lon_match = pick(lon_candidates, prefer_plant=True)
    return lat_match, lon_match

def fetch_egrid_from_excel(study_geom):
    # If you already downloaded the file in your previous step, reuse it:
    # Otherwise, you can still download directly by pasting the current URL you used.
    egrid_xlsx = RAW_DIR / "egrid" / "egrid2023_data_rev2.xlsx"  # adjust if needed
    if not egrid_xlsx.exists():
        raise FileNotFoundError(f"eGRID Excel not found at {egrid_xlsx}. Download it first.")

    xls = pd.ExcelFile(egrid_xlsx)
    # try to find plant sheet
    sheet_candidates = [s for s in xls.sheet_names if s.strip().lower() in ("plnt","plant","plants","plant data","plants data")]
    sheet_name = sheet_candidates[0] if sheet_candidates else xls.sheet_names[0]
    print(f"\nReading sheet: {sheet_name}")
    df = pd.read_excel(egrid_xlsx, sheet_name=sheet_name)

    # Try robust detection
    lat_col, lon_col = _guess_lat_lon_columns(df.columns)
    if lat_col is None or lon_col is None:
        print("\nCould not auto-detect latitude/longitude.")
        print("Available columns:")
        for c in df.columns:
            print(" -", c)
        # Minimal interactive fallback in Colab:
        # (type the exact column names as they appear above)
        lat_col = input("\nType the LATITUDE column name exactly as shown: ").strip()
        lon_col = input("Type the LONGITUDE column name exactly as shown: ").strip()
        if lat_col not in df.columns or lon_col not in df.columns:
            raise RuntimeError("Provided column names not found in the sheet. Please check spelling/case.")

    print(f"Using latitude='{lat_col}', longitude='{lon_col}'")

    # Coerce to numeric
    df[lat_col] = pd.to_numeric(df[lat_col], errors="coerce")
    df[lon_col] = pd.to_numeric(df[lon_col], errors="coerce")
    g = df.dropna(subset=[lat_col, lon_col]).copy()

    # Build GeoDataFrame
    gdf = gpd.GeoDataFrame(
        g,
        geometry=gpd.points_from_xy(g[lon_col], g[lat_col], crs="EPSG:4326")
    ).to_crs(study_geom.crs)

    # Clip to study area (+ buffer)
    gdf = gdf[gdf.geometry.within(study_geom.buffer(50_000).geometry.iloc[0])].copy()

    out = OUT_DIR / "egrid_plants_seattle.gpkg"
    out.parent.mkdir(parents=True, exist_ok=True)
    gdf.to_file(out, driver="GPKG")
    print("Saved plant points near Seattle →", out)
    return out


In [18]:
egrid_out = fetch_egrid_from_excel(study)


Reading sheet: Contents

Could not auto-detect latitude/longitude.
Available columns:
 - Unnamed: 0
 - Unnamed: 1
 - Unnamed: 2
 - Unnamed: 3
 - Unnamed: 4
 - Unnamed: 5
 - Unnamed: 6
 - Unnamed: 7
 - Unnamed: 8
 - Unnamed: 9
 - Unnamed: 10
 - Unnamed: 11
 - Unnamed: 12
 - Unnamed: 13
 - Unnamed: 14
 - Unnamed: 15
 - Unnamed: 16


KeyboardInterrupt: Interrupted by user

In [19]:
import pandas as pd
from pathlib import Path

egrid_xlsx = RAW_DIR / "egrid" / "egrid2023_data_rev2.xlsx"  # adjust if your filename differs
xls = pd.ExcelFile(egrid_xlsx)
print(xls.sheet_names)


['Contents', 'UNT23', 'GEN23', 'PLNT23', 'ST23', 'BA23', 'SRL23', 'NRL23', 'US23', 'GGL23', 'DEMO23']


In [20]:
import pandas as pd
from pathlib import Path

egrid_xlsx = RAW_DIR / "egrid" / "egrid2023_data_rev2.xlsx"  # adjust if your filename differs
xls = pd.ExcelFile(egrid_xlsx)
print("Sheets:", xls.sheet_names)

plnt = pd.read_excel(egrid_xlsx, sheet_name="PLNT23", nrows=5)
print("\nPLNT23 columns (first 5 rows loaded):")
for c in plnt.columns:
    print(" -", c)


Sheets: ['Contents', 'UNT23', 'GEN23', 'PLNT23', 'ST23', 'BA23', 'SRL23', 'NRL23', 'US23', 'GGL23', 'DEMO23']

PLNT23 columns (first 5 rows loaded):
 - Plant file sequence number
 - Data Year
 - Plant state abbreviation
 - Plant name
 - DOE/EIA ORIS plant or facility code
 - Plant transmission or distribution system owner name
 - Plant transmission or distribution system owner ID
 - Utility name
 - Utility ID
 - Plant-level sector
 - Balancing Authority Name
 - Balancing Authority Code
 - NERC region acronym
 - eGRID subregion acronym
 - eGRID subregion name
 - Plant associated ISO/RTO Territory
 - Plant FIPS state code
 - Plant FIPS county code
 - Plant county name
 - Plant latitude
 - Plant longitude
 - CAPD Program Flag
 - Number of units
 - Number of generators
 - Plant primary fuel
 - Plant primary fuel category
 - Flag indicating if the plant burned or generated any amount of coal
 - Plant capacity factor
 - Plant nameplate capacity (MW)
 - Nonbaseload Factor
 - Biogas/ biomass p

In [21]:
import re
import numpy as np
import geopandas as gpd
import pandas as pd
from pathlib import Path

def _norm(s: str) -> str:
    return re.sub(r'[^a-z0-9]', '', s.lower())

def _find_lat_lon_cols(cols):
    # Try a bunch of common patterns (with/without "plant", underscores, spaces, etc.)
    norm = {_norm(c): c for c in cols}
    keys = list(norm.keys())

    lat_patts = [
        r'(?:^|.*)plantlat(?:itude)?(?:.*)$',
        r'(?:^|.*)lat(?:itude)?(?:.*)$',
        r'(?:^|.*)ycoord(?:.*)$', r'(?:^|.*)\by\b'
    ]
    lon_patts = [
        r'(?:^|.*)plantlon(?:g|gitude)?(?:.*)$',
        r'(?:^|.*)lon(?:g|gitude)?(?:.*)$',
        r'(?:^|.*)\blng\b(?:.*)$',
        r'(?:^|.*)xcoord(?:.*)$', r'(?:^|.*)\bx\b'
    ]

    def pick(patts):
        # prefer matches that include 'plant' in the normalized key
        for pat in patts:
            for k in keys:
                if re.match(pat, k) and 'plant' in k:
                    return norm[k]
        for pat in patts:
            for k in keys:
                if re.match(pat, k):
                    return norm[k]
        return None

    return pick(lat_patts), pick(lon_patts)

def fetch_egrid_from_excel_plnt23(study_geom):
    egrid_xlsx = RAW_DIR / "egrid" / "egrid2023_data_rev2.xlsx"  # adjust if needed
    if not egrid_xlsx.exists():
        raise FileNotFoundError(f"Excel not found at {egrid_xlsx}")

    df = pd.read_excel(egrid_xlsx, sheet_name="PLNT23")  # <-- explicit sheet name
    # Try to auto-detect lat/lon
    lat_col, lon_col = _find_lat_lon_cols(df.columns)

    if lat_col is None or lon_col is None:
        print("Could not auto-detect latitude/longitude columns.")
        print("Available columns:")
        for c in df.columns:
            print(" -", c)
        # small interactive fallback
        lat_col = input("\nType the LATITUDE column name exactly as shown: ").strip()
        lon_col = input("Type the LONGITUDE column name exactly as shown: ").strip()
        if lat_col not in df.columns or lon_col not in df.columns:
            raise RuntimeError("Provided names do not exist in PLNT23. Please check spelling/case.")

    print(f"Using latitude='{lat_col}', longitude='{lon_col}'")

    # Coerce to numeric and drop nulls
    df[lat_col] = pd.to_numeric(df[lat_col], errors="coerce")
    df[lon_col] = pd.to_numeric(df[lon_col], errors="coerce")
    df2 = df.dropna(subset=[lat_col, lon_col]).copy()

    # Create GeoDataFrame and clip to Seattle (+50 km buffer)
    gdf = gpd.GeoDataFrame(
        df2,
        geometry=gpd.points_from_xy(df2[lon_col], df2[lat_col], crs="EPSG:4326")
    ).to_crs(study_geom.crs)

    gdf = gdf[gdf.geometry.within(study_geom.buffer(50_000).geometry.iloc[0])].copy()

    out = OUT_DIR / "egrid_plants_seattle.gpkg"
    out.parent.mkdir(parents=True, exist_ok=True)
    gdf.to_file(out, driver="GPKG")
    print("Saved plant points near Seattle →", out)
    return out

egrid_out = fetch_egrid_from_excel_plnt23(study)


Using latitude='Plant latitude', longitude='Plant longitude'
Saved plant points near Seattle → /content/UHI_Project/data/processed/egrid_plants_seattle.gpkg


In [25]:
def fetch_nlcd(study_geom):
    # If MRLC rotates links, use the MRLC Viewer to get fresh URLs and swap them here.
    nlcd_imp_zip = "https://www.mrlc.gov/sites/default/files/2021-06/NLCD_2019_Impervious_L48_20210604.zip"
    nlcd_lc_zip  = "https://www.mrlc.gov/sites/default/files/2021-06/NLCD_2019_Land_Cover_L48_20210604.zip"

    imp_zip = RAW_DIR / "NLCD" / "NLCD_2019_Impervious_L48_20210604.zip"
    lc_zip  = RAW_DIR / "NLCD" / "NLCD_2019_Land_Cover_L48_20210604.zip"
    imp_dir = RAW_DIR / "NLCD" / "impervious"
    lc_dir  = RAW_DIR / "NLCD" / "landcover"

    print("\nDownloading NLCD Impervious…")
    download_file(nlcd_imp_zip, imp_zip); unzip_to(imp_zip, imp_dir)

    print("Downloading NLCD Land Cover…")
    download_file(nlcd_lc_zip, lc_zip); unzip_to(lc_zip, lc_dir)

    imp_tif = next((p for p in imp_dir.rglob("*.img")), None) or next((p for p in imp_dir.rglob("*.tif")), None)
    lc_tif  = next((p for p in lc_dir.rglob("*.img")), None) or next((p for p in lc_dir.rglob("*.tif")), None)
    if not imp_tif or not lc_tif:
        raise FileNotFoundError("Could not find NLCD geotiffs after unzip.")

    imp_out = OUT_DIR / "NLCD" / "nlcd_2019_impervious_seattle.tif"
    lc_out  = OUT_DIR / "NLCD" / "nlcd_2019_landcover_seattle.tif"
    imp_out.parent.mkdir(parents=True, exist_ok=True)

    save_clip_raster(imp_tif, study_geom, imp_out)
    save_clip_raster(lc_tif,  study_geom, lc_out)
    print("Saved NLCD clips →", imp_out.name, lc_out.name)
    return imp_out, lc_out

nlcd_imp_tif, nlcd_lc_tif = fetch_nlcd(study)


HTTPError: 404 Client Error: Not Found for url: https://www.mrlc.gov/sites/default/files/2021-06/NLCD_2019_Impervious_L48_20210604.zip

In [22]:
# Study area boundary
study_fp = OUT_DIR / "boundaries" / "seattle_study_area.gpkg"
study = gpd.read_file(study_fp)
study = study.to_crs(3857)  # Web Mercator for quick visuals
study


,geometry
0,"POLYGON ((-13525318.131 5958411.92, -13525318...."


In [23]:
# eGRID plant points (already filtered to Seattle vicinity)
plants_fp = OUT_DIR / "egrid_plants_seattle.gpkg"
plants = gpd.read_file(plants_fp).to_crs(study.crs)
print(plants.shape)
plants.head(3)


(36, 151)


,Plant file sequence number,Data Year,Plant state abbreviation,Plant name,DOE/EIA ORIS plant or facility code,Plant transmission or distribution system owner name,Plant transmission or distribution system owner ID,Utility name,Utility ID,Plant-level sector,...,Plant other fossil generation percent (resource mix),Plant other unknown / purchased fuel generation percent (resource mix),Plant total nonrenewables generation percent (resource mix),Plant total renewables generation percent (resource mix),Plant total nonrenewables other unknown/purchased generation percent (resource mix),Plant total nonhydro renewables generation percent (resource mix),Plant total combustion generation percent (resource mix),Plant total noncombustion generation percent (resource mix),Plant total noncombustion other unknown/purchased generation percent (resource mix),geometry
0,12132,2023,WA,Alder,3913,City of Tacoma - (WA),18429,City of Tacoma - (WA),18429,Electric Utility,...,0,0,0,1,0,0,0,1,0,POINT (-13615509.183 5909750.056)
1,12133,2023,WA,Arlington Microgrid,64446,PUD 1 of Snohomish County,17470,PUD No 1 of Snohomish County,17470,Electric Utility,...,0,0,0,1,0,1,0,1,0,POINT (-13597698.064 6132870.389)
2,12137,2023,WA,Black Creek,54860,Puget Sound Energy Inc,15500,Black Creek Hydro Inc,1784,IPP Non-CHP,...,0,0,0,1,0,0,0,1,0,POINT (-13548752.777 6032257.335)


In [24]:
# NLCD landcover + impervious (clipped to Seattle area)
nlcd_lc_fp  = OUT_DIR / "NLCD" / "nlcd_2019_landcover_seattle.tif"
nlcd_imp_fp = OUT_DIR / "NLCD" / "nlcd_2019_impervious_seattle.tif"

# Load as xarray DataArrays (rioxarray)
nlcd_lc  = rxr.open_rasterio(nlcd_lc_fp).squeeze().rio.write_crs("EPSG:3857", inplace=True)
nlcd_imp = rxr.open_rasterio(nlcd_imp_fp).squeeze().rio.write_crs("EPSG:3857", inplace=True)

nlcd_lc, nlcd_imp


RasterioIOError: /content/UHI_Project/data/processed/NLCD/nlcd_2019_landcover_seattle.tif: No such file or directory